In [1]:
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.oauth2.credentials import Credentials
from google.oauth2 import service_account
from google.auth.transport.requests import Request
import os.path
import json
import pickle
import requests

def oauth():
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets', 
              'https://www.googleapis.com/auth/script.projects', 'https://www.googleapis.com/auth/drive', 'https://www.googleapis.com/auth/drive.file'
              ]
    creds_filename = 'test-daebong-service-account.json'  # 서비스 계정 파일 경로를 지정합니다.

    # 서비스 계정 파일을 사용하여 인증 정보를 로드합니다.
    creds = service_account.Credentials.from_service_account_file(creds_filename, scopes=SCOPES)
    return creds

def oauthByWeb():
    # 필요한 스코프를 지정합니다.
    SCOPES = [
        'https://www.googleapis.com/auth/spreadsheets',
        'https://www.googleapis.com/auth/script.projects',
        'https://www.googleapis.com/auth/drive',
        'https://www.googleapis.com/auth/drive.file'
    ]
    creds = None
    # 'token.json' 파일이 존재하면, 저장된 인증 정보를 불러옵니다.
    if os.path.exists('token.json'):
        with open('token.json', 'rb') as token:
            creds = pickle.load(token)

    # 저장된 인증 정보가 없거나, 유효하지 않은 경우 새로운 인증을 진행합니다.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            creds_filename = 'oauth-new-daebong.json'
            flow = InstalledAppFlow.from_client_secrets_file(creds_filename, SCOPES)
            creds = flow.run_local_server(port=8080)
        # 새로운 인증 정보를 'token.json'에 저장합니다.
        with open('token.json', 'wb') as token:
            pickle.dump(creds, token)

    return creds



In [12]:
import requests
import json



access_key = "65f3caa27ae3e0f1c64f"
access_secret = "5a39495fffc8c62e228d76a68bda078f"


def upsert_user(user_id, name, phone_number, email, sheet_url):
    url = f'https://api.channel.io/open/v5/users/@{user_id}'

    payload = {
    "profile": {
        "name": name,
        "mobileNumber": f'+82{phone_number}',
        "email": email,
        "description": sheet_url,
        },
    }

    headers = {
        "Content-Type": "application/json",
        "x-access-key": access_key,
        "x-access-secret": access_secret
    }

    response = requests.put(url, data=json.dumps(payload), headers=headers)

    print(response.text)

In [7]:
import requests
import json



access_key = "65f3caa27ae3e0f1c64f"
access_secret = "5a39495fffc8c62e228d76a68bda078f"


def upsert_user(user_id, name, phone_number, email, sheet_url):
    url = f'https://api.channel.io/open/v5/users/@{user_id}'

    payload = {
    "profile": {
        "name": name,
        "mobileNumber": f'+82{phone_number}',
        "email": email,
        "description": sheet_url,
        },
        "tags":["자동화후"],
    }

    headers = {
        "Content-Type": "application/json",
        "x-access-key": access_key,
        "x-access-secret": access_secret
    }

    response = requests.put(url, data=json.dumps(payload), headers=headers)

    print(response.text)

In [8]:
def get_sheet_data(service, spreadsheet_id, range_name):
    """특정 스프레드시트의 지정된 범위에 있는 데이터를 가져옵니다."""
    sheet = service.spreadsheets()
    result = sheet.values().get(spreadsheetId=spreadsheet_id, range=range_name).execute()
    values = result.get('values', [])

    # if not values:
    #     print('No data found.')
    # else:
    #     for row in values:
    #         # Row 데이터를 출력합니다. 필요에 따라 여기서 다른 작업을 수행할 수 있습니다.
    #         print(row)

    return values
    

In [10]:
signup_spreadsheets_id = "1Ajz2g-pChLSoies5umhPMcgTp5m2MIjhljJOviPlWCU"
RANGE_NAME = '셀러_전화번호있는_셀러!A48:N126'  # 예를 들어 Sheet1의 K2부터 K 끝까지

if __name__ == "__main__":
    creds = oauthByWeb()
    service_sheets = build('sheets', 'v4', credentials=creds)

    spreadsheet_ids = get_sheet_data(service_sheets, signup_spreadsheets_id, RANGE_NAME)

    print(spreadsheet_ids)
    for value in spreadsheet_ids[1:]:

        user_name = value[3]
        user_phone = value[9]
        user_id = value[11]
        user_url = value[12]
        url_split = user_url[8:]
        user_email = value[10]

#         print(
# user_name,
# user_phone,
# user_id,
# user_url,
# url_split,
# user_email

#         )

        upsert_user(user_id, user_name, user_phone, user_email, url_split)

[['자동화이후', '226', '농부의마음', '226_농부의마음', '', '', '', '', '', '1036445918', 'kjj890509@gmail.com', '1-J6V0nLHQlxLA8PbTiTXaGt08VlqGv5fo3kfaH35FJE', 'https://docs.google.com/spreadsheets/d/1-J6V0nLHQlxLA8PbTiTXaGt08VlqGv5fo3kfaH35FJE/edit#gid=1464675013'], ['', '227', '새콤달콤써니네', '227_새콤달콤써니네', '', '', '', '', '', '1062244640', 'hash861016@gmail.com', '14KS38kY_hZD2061EmIaAm3cA3Rw9R2mEDoIczxdk3sQ', 'https://docs.google.com/spreadsheets/d/14KS38kY_hZD2061EmIaAm3cA3Rw9R2mEDoIczxdk3sQ'], ['', '228', '예스마켓', '228_예스마켓', '', '', '', '', '', '1088983526', 'jyya3526@gmail.com', '1hrmwW1aznOQJewXDlyCsfCqEvsTjh_BXXVSZAqwH2r0', 'https://docs.google.com/spreadsheets/d/1hrmwW1aznOQJewXDlyCsfCqEvsTjh_BXXVSZAqwH2r0'], ['', '229', '디제이 유통', '229_디제이 유통', '', '', '', '', '', '1036514834', 'cjfals100486@gmail.com', '1aEZw4wsj1HXCuxvcc5mnvJ0UI8Qkuo05-zmYPzblSWc', 'https://docs.google.com/spreadsheets/d/1aEZw4wsj1HXCuxvcc5mnvJ0UI8Qkuo05-zmYPzblSWc/edit#gid=1464675013'], ['', '230', '문제우', '230_문제우', '', '', '

In [16]:
def get_sheet_array_id_number(service, spreadsheet_id, range_name):
    """특정 스프레드시트의 지정된 범위에 있는 데이터를 가져옵니다."""
    sheet = service.spreadsheets()
    result = sheet.values().get(spreadsheetId=spreadsheet_id, range=range_name).execute()
    values = result.get('values', [])

    # data = []

    # if not values:
    #     print('No data found.')
    # else:
    #     for row in values:
    #         # Row 데이터를 출력합니다. 필요에 따라 여기서 다른 작업을 수행할 수 있습니다.
    #         print(row)
    #         data.append(row[1], row[11])

    return values
    

In [58]:
import requests
import json



access_key = "65f3caa27ae3e0f1c64f"
access_secret = "5a39495fffc8c62e228d76a68bda078f"


def update_event_user(user_id, event_url, event_tag_list, user_name, user_phone, user_email, docs_url):


    url = f'https://api.channel.io/open/v5/users/@{user_id}'

    print("event_tag_list", event_tag_list)

    payload = {
    "profile": {
         "name": user_name,
         "mobileNumber": f'+82{user_phone}',
        "eventlink": event_url,
        "description":docs_url,
        },
        "tags":event_tag_list,
    }

    if user_email: 
        payload = {
        "profile": {
             "name": user_name,
            "mobileNumber": f'+82{user_phone}',
            "email": user_email,
            "eventlink": event_url,
            "description":docs_url,
            },
            "tags":event_tag_list,
        }

    headers = {
        "Content-Type": "application/json",
        "x-access-key": access_key,
        "x-access-secret": access_secret
    }

    response = requests.put(url, data=json.dumps(payload), headers=headers)

    print(response.text)

In [60]:
# signup_spreadsheets_id = "1Ajz2g-pChLSoies5umhPMcgTp5m2MIjhljJOviPlWCU"
update_sheet_id = "1UmrhVFYSzwfqu2g0BLbQlnVkzN9AXgINtwoGBgLEzUc"
RANGE_NAME = '이벤트지급!A2:H36'  # 예를 들어 Sheet1의 K2부터 K 끝까지

if __name__ == "__main__":
    creds = oauthByWeb()
    service_sheets = build('sheets', 'v4', credentials=creds)

    spreadsheet_ids = get_sheet_array_id_number(service_sheets, update_sheet_id, RANGE_NAME)

    print(spreadsheet_ids)
    for value in spreadsheet_ids[2:]:

        # user_name = value[3]
        # user_phone = value[9]

        if value[2] == "미입점":
            continue
        
        print(value)

        user_id = value[4]
        event_url = value[3][8:len(value[3])]
        user_number = value[2]
        event_tag = value[1]

        user_name = f'{value[2]}_{value[5]}'
        user_phone = value[0]

        user_email = ""
        docs_url = value[7][8:len(value[7])]

        if len(value) == 7:
            user_email = value[6]
        



        array_event_tag = []

        if int(user_number) >= 226:
            array_event_tag = ["자동화후", event_tag]
        else:
            array_event_tag = [event_tag]


        print(
user_id,
event_url,
user_number,
array_event_tag,
user_name,
user_phone,
user_email,
docs_url,
        )

        

        update_event_user(user_id, event_url, array_event_tag, user_name, user_phone, user_email, docs_url )

[['01036445918', '오렌지', '226', 'https://drive.google.com/drive/folders/1J_snfmfIzKf4BkUKIYDrDQLuuXE5pDOm?usp=drive_link', '1-J6V0nLHQlxLA8PbTiTXaGt08VlqGv5fo3kfaH35FJE', '농부의마음', 'kjj890509@gmail.com', 'https://docs.google.com/spreadsheets/d/1-J6V0nLHQlxLA8PbTiTXaGt08VlqGv5fo3kfaH35FJE/edit#gid=1464675013'], ['01067887067', '방울토마토', '198', 'https://drive.google.com/drive/folders/1j0Rn4yTodwvuYMadcE6PX8Sbqq5bSd62?usp=drive_link', '15sKc1_sjgo4WV8LIF_M68RqctBKJeUcO4FVh1cv-mCs', '토박이상회', '', 'https://docs.google.com/spreadsheets/d/15sKc1_sjgo4WV8LIF_M68RqctBKJeUcO4FVh1cv-mCs'], ['01023949182', '방울토마토', '105', 'https://drive.google.com/drive/folders/1j0Rn4yTodwvuYMadcE6PX8Sbqq5bSd62?usp=drive_link', '1FPRUzUjvrbZ7wscG9btA97Tk4f_1_1nk3G-RLUPJ--k', '달콤이창고', '', 'https://docs.google.com/spreadsheets/d/1FPRUzUjvrbZ7wscG9btA97Tk4f_1_1nk3G-RLUPJ--k'], ['01075101608', '짭짤이 토마토(조O호 농부님)', '171', 'https://drive.google.com/drive/folders/1bnGhcwExbmIywScEKuIeTTVIrnzgvkES?usp=drive_link', '1ZOX9E8BaYZ

In [43]:
if __name__ == "__main__":
    update_event_user("1MuQVZJ5fK6JCtYyr-fxy3hinWgV4SlwAk2fG-MLRrGA", "drive.google.com/drive/folders/1J_snfmfIzKf4BkUKIYDrDQLuuXE5pDOm?usp=drive_link", ["자동화후", "방울토마토"])

event_tag_list ['자동화후', '방울토마토']
{"user":{"id":"65f8235a1453a2e183bc","channelId":"164847","memberId":"1MuQVZJ5fK6JCtYyr-fxy3hinWgV4SlwAk2fG-MLRrGA","type":"member","name":"test_백민기","mobileNumberQualified":true,"emailQualified":true,"profile":{"referrer":"https://www.google.co.kr/","mobileNumber":"+821052371951","name":"test_백민기","description":"docs.google.com/spreadsheets/d/1MuQVZJ5fK6JCtYyr-fxy3hinWgV4SlwAk2fG-MLRrGA/edit#gid=1464675013","email":"toyoalsrl@gmail.com","eventlink":"drive.google.com/drive/folders/1J_snfmfIzKf4BkUKIYDrDQLuuXE5pDOm?usp=drive_link"},"tags":["자동화후","방울토마토"],"alert":0,"unread":0,"blocked":false,"unsubscribeEmail":false,"unsubscribeTexting":false,"hasChat":true,"hasPushToken":false,"lastSeenAt":1710830021839,"createdAt":1710760794083,"updatedAt":1711009509718,"version":35,"managedKey":1710830021839,"named":true,"member":true,"email":"toyoalsrl@gmail.com","avatarUrl":"https://cf.channel.io/avatar/emoji/popcorn.f392e0.png","mobileNumber":"+821052371951","syste

In [37]:
import requests
import json



access_key = "65f3caa27ae3e0f1c64f"
access_secret = "5a39495fffc8c62e228d76a68bda078f"


def upsert_event_user_test(user_id):


    url = f'https://api.channel.io/open/v5/users/@{user_id}'


    payload = {
    "profile": {
        },
        "tags":['테스트1'],
    }

    headers = {
        "Content-Type": "application/json",
        "x-access-key": access_key,
        "x-access-secret": access_secret
    }

    response = requests.put(url, data=json.dumps(payload), headers=headers)

    print(response.text)


def update_event_user_test(user_id, eventTagNameList, eventDetailTagList, eventlink):


    url = f'https://api.channel.io/open/v5/users/{user_id}'


    payload = {
        "profile": {
            "eventTagNameList": eventTagNameList,
            "eventDetailTagList":eventDetailTagList,
            "eventlink":eventlink, 
        }
    }

    headers = {
        "Content-Type": "application/json",
        "x-access-key": access_key,
        "x-access-secret": access_secret
    }

    response = requests.patch(url, data=json.dumps(payload), headers=headers)

    print(response.text)

def get_user_test(memeber_id):


    url = f'https://api.channel.io/open/v5/users/@{memeber_id}'


    payload = {
        "tags":['테스트2'],
    }

    headers = {
        "Content-Type": "application/json",
        "x-access-key": access_key,
        "x-access-secret": access_secret
    }

    response = requests.get(url, headers=headers)

    print(response.json()["user"]["profile"])
    print(response.json()["user"]["profile"]["eventTagNameList"])
    print(response.json()["user"]["profile"]["eventDetailTagList"])
    print(response.json()["user"]["profile"]["eventlink"])

    return response.json()["user"]["profile"] 

    


In [45]:
if __name__ == "__main__":
    # user_id = "1RAnWFmNvlv2VUk_biNbpUyhasr6o2Gt5HXZE6sDs"
    user_id = "1RAnWFmNvlv2VUk_biNbpUyhasr6o2Gt5HXZEk216sDs"

    sheet_eventTagNameList = []
    sheet_eventDetailTagList = []
    sheet_eventlink = ""

    try:
        user_profile_data = get_user_test(user_id)

        if "eventTagNameList" in user_profile_data:
            eventTagNameList = user_profile_data["eventTagNameList"] + sheet_eventTagNameList
        else:
            eventTagNameList = "eventTagNameList"

        if "eventDetailTagList" in user_profile_data:
            eventDetailTagList = user_profile_data["eventDetailTagList"] + sheet_eventDetailTagList
        else:
            eventDetailTagList = "eventDetailTagList"
        
      
        eventlink = sheet_eventlink

        upsert_event_user_test(user_id, eventTagNameList, eventlink)
        
        

    except Exception as e:
        print(e, "가 없습니다.")
        upsert_event_user_test("1RAnWFmNvlv2VUk_biNbpUyhasr6o2Gt5HXZEk216sDs")
    # update_event_user_test("1RAnWFmNvlv2VUk_biNbpUyhasr6o2Gt5HXZEk216sDs")
    

SyntaxError: invalid syntax (3029747889.py, line 13)